In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
torch.cuda.is_available()

True

In [3]:
# Load data from solcast.csv
df = pd.read_csv('solcast.csv')

# Define the features (X) and label (y)
X = df[['air_temp', 'azimuth', 'cloud_opacity']]
y = df['ghi']

# Split into train and test (80% train, 20% test)
train_size = 0.8
train_samples = int(train_size * len(df))

X_train, X_test = X[:train_samples], X[train_samples:]
y_train, y_test = y[:train_samples], y[train_samples:]


In [4]:

# Print the first few rows of features and label to verify
print("Features:")
print(X_train.head())
print("\nLabel:")
print(y_train.head())


Features:
   air_temp  azimuth  cloud_opacity
0        25     -143           41.0
1        25     -115           37.0
2        25     -109           33.0
3        25     -108           29.5
4        24     -109           26.0

Label:
0    0
1    0
2    0
3    0
4    0
Name: ghi, dtype: int64


In [5]:
# class SolarLSTM(nn.Module):
#     def __init__(self, input_size, hidden_size, output_size, sequence_length):
#         super(SolarLSTM, self).__init__()
#         self.lstm = nn.LSTM(input_size, hidden_size)
#         self.label = nn.Linear(hidden_size, output_size)
#         self.sequence_length = sequence_length

#     def forward(self, input):
#         h_0 = Variable(torch.zeros(1, input.size(0), self.hidden_size).cuda())
#         c_0 = Variable(torch.zeros(1, input.size(0), self.hidden_size).cuda())
#         output, (final_hidden_state, final_cell_state) = self.lstm(input, (h_0, c_0))
#         return self.label(final_hidden_state[-1])


In [6]:
# Normalize the data
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)
X_test_normalized = scaler.transform(X_test)

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train_normalized, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_normalized, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

In [7]:
# Define the LSTM model
class SolarLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SolarLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input):
        output, _ = self.lstm(input)
        return self.fc(output[-1])  # Predict the GHI directly

# Initialize hyperparameters
input_size = 3  # Number of features (azimuth, air temperature, cloud opacity)
hidden_size = 64
output_size = 1  # Predicting GHI

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize model, optimizer, and loss function
model = SolarLSTM(input_size, hidden_size, output_size)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()


In [12]:
print(batch_X)

tensor([[ 0.3184,  0.6099, -0.8596],
        [-0.6098, -1.0487, -0.8596],
        [ 0.3184,  1.1419, -0.8596],
        [ 1.2465, -1.8206,  0.5536],
        [ 1.2465,  1.1731,  0.8998],
        [ 0.7824, -0.7253, -0.4180],
        [-0.1457,  0.6307,  1.0200],
        [-1.5379,  1.6530, -0.8596],
        [-1.0738, -0.7358,  1.6135],
        [ 3.1027,  0.8393,  1.2461],
        [-0.1457,  0.7350, -0.8596],
        [-1.5379,  1.1523, -0.8596],
        [ 0.3184,  0.5994, -0.8596],
        [-1.5379,  0.8602, -0.4568],
        [ 1.2465, -1.8311, -0.2731],
        [ 0.3184,  0.5681, -0.8596],
        [-0.1457,  0.1822, -0.8596],
        [ 0.7824,  0.7142, -0.8596],
        [-0.6098, -0.6419, -0.8596],
        [-1.0738, -0.8818, -0.8596],
        [-0.6098,  0.5473, -0.8596],
        [-1.5379, -1.0800, -0.8596],
        [-0.6098,  1.1419, -0.0223],
        [ 0.3184,  1.1001, -0.1813],
        [-1.0738, -1.1322, -0.8596]])


In [8]:

# Training loop
for epoch in range(10):  # Adjust the number of epochs
    model.train()
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/10], Loss: {loss.item()}")


c:\Users\janrh\OneDrive - University of the Philippines\Acads\4TH YEAR (23-24)\2ND SEM\EE 199\eee-capstone\env3.10\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
c:\Users\janrh\OneDrive - University of the Philippines\Acads\4TH YEAR (23-24)\2ND SEM\EE 199\eee-capstone\env3.10\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([25])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/10], Loss: 44560.88671875
Epoch [2/10], Loss: 78507.6328125
Epoch [3/10], Loss: 51930.5
Epoch [4/10], Loss: 48191.375
Epoch [5/10], Loss: 62567.453125
Epoch [6/10], Loss: 60801.1015625
Epoch [7/10], Loss: 36394.796875
Epoch [8/10], Loss: 69941.3515625
Epoch [9/10], Loss: 58128.2890625
Epoch [10/10], Loss: 46193.49609375


In [10]:

# Validation and testing (similar process)
# ...

# Once trained, use the model for solar energy predictions
# For example:
with torch.no_grad():
    model.eval()
    test_predictions = model(X_test_tensor)
    # Evaluate performance metrics (e.g., RMSE, MAE, etc.)

In [11]:
test_predictions

tensor([170.2512])

---------------------**Activate API Endpoint** before proceeding-------------------------

In [ ]:
import requests


In [ ]:

# input_data = {
#     "input": [
#         [0.1, 0.2, 0.3],
#         [0.4, 0.5, 0.6],
#         [0.1, 0.2, 0.3],
#         [0.4, 0.5, 0.6],
#         [0.1, 0.2, 0.3],
#         [0.4, 0.5, 0.6],
#         [0.1, 0.2, 0.3],
#         [0.4, 0.5, 0.6],
#         [0.1, 0.2, 0.3],
#         [0.4, 0.5, 0.6],
        
#     ]
# }


In [ ]:

# response = requests.post('http://localhost:5000/predict', json=input_data).json()
# # print(response)

# threshold = 0.5
# RBP = response["output"]

# print("RBP =", RBP)
# if RBP >= threshold:
#     print("WARNING! Impending robbery.")


In [ ]:
for i in range(int(n_samples*0.15)):
    input_data = {
    "input":  X_test_scaled.tolist()[i] 
}
    response = requests.post('http://localhost:5000/predict', json=input_data).json()

    threshold = 0.5
    RBP = response["output"]

    print(i, "RBP =", RBP, "| actual RBP =", y_test.tolist()[i])
    if RBP >= threshold:
        print("WARNING! Impending robbery.")

In [ ]:
# study first doen lstm
# work on dynamic input
# try solar dataset